# Pure State Decomposition

<a target="_blank" href="https://colab.research.google.com/github/numqi/numqi/blob/main/docs/application/resource/pure-state-decomp.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

reference: Unified Framework for Calculating Convex Roof Resource Measures [arxiv-link](https://arxiv.org/abs/2406.19683)

Many quantum phenomena have transitioned from being of purely theoretical interest to serving as invaluable resources in quantum information processing tasks. Convex roof extension is a widely used measures for quantifying the convex resource theories like entanglement, coherence and magic states. Convex roof extension begins with a measure of pure
states and then extends to mixed states:

$$ R(\rho)=\min_{\left\{ p_{i},|\psi_{i}\rangle\right\} }\sum_{i}p_{i}R\left(|\psi_{i}\rangle\right), $$

where the minimization is taken over all possible pure state decompositions of the given mixed state $\rho$ satisfying

$$ \rho=\sum_{i}p_{i}|\psi_{i}\rangle\langle\psi_{i}| $$

In this notebook, we will discuss how to perform the pure state decomposition, and how to do the minimization over all possible pure state decompositions.

In [ ]:
import numpy as np
import torch

try:
    import numqi
except ImportError:
    %pip install numqi
    import numqi

np_rng = np.random.default_rng()

## Stiefel manifold

Before we start, let's introduce the Stiefel manifold. The Stiefel manifold $St(n,r)$ is the set of $n$-by-$r$ with complex matrices with orthogonal contraints:

$$ \mathrm{St}(n,r)=\left\{ A\in\mathbb{C}^{n\times r}:A^{\dagger}A=I_{r}\right\}. $$

The element $A$ is called a Stiefel matrix. It's different from the unitary matrices which requires orthogonal contraints on both sides:

$$ A\in U(n)\Rightarrow A^{\dagger}A=AA^{\dagger}=I_n. $$

One can think of the Stiefel matrix as the first $r$ columns of a uniatry matrix. For example, stacking two Bell states makes a Stiefel matrix of size $4\times 2$:

$$ \left[\left|\Phi^{+}\right\rangle ,\left|\Psi^{+}\right\rangle \right]=\frac{1}{\sqrt{2}}\left[\begin{array}{cc}
1&0\\
0 & 1\\
0 & 1\\
1 & 0
\end{array}\right]\in\mathrm{St}(4,2) $$

In [ ]:
A = np.stack([numqi.state.Bell(0), numqi.state.Bell(2)], axis=1)
print(A)

Random Stiefel matrix can be generated:

In [ ]:
A = numqi.random.rand_Stiefel_matrix(5, 3, iscomplex=True)
print('A\n', np.around(A, 3))

Let's check $A^{\dagger}A=I_{r}$:

In [ ]:
print('A^dag A=\n', np.around(A.conj().T @ A, 3))

Only when $n=r$ (unitary matrix), $AA^{\dagger}=I_{n}$ is true.

In [ ]:
A = numqi.random.rand_Stiefel_matrix(4, 2, iscomplex=True)
print('A in St(4,2): A^dag A=\n', np.around(A@A.T.conj(), 3))

A = numqi.random.rand_Stiefel_matrix(4, 4, iscomplex=True)
print('\nA in St(4,4): A^dag A=\n', np.around(A@A.T.conj(), 3))

## Pure state decomposition and Stiefel matrix

In this part, we will show how to perform the pure state decomposition with the help of Stiefel matrix. Let's try its inverse problem first to make familiar with this decomposition:

> Given the probability distribution $\{p_i\}$ and the corresponding pure states $\{|\psi_i\rangle\}$, how to construct the density matrix $\rho$

The answer is simple: just follow the formula $\rho=\sum_i p_i|\psi_i\rangle\langle\psi_i|$.

In [ ]:
dim = 3
num_state = 5
prob_list = numqi.random.rand_discrete_probability(num_state)
psi_list = numqi.random.rand_haar_state(dim, batch_size=num_state)

mat0 = sum(x*y.reshape(-1,1)*y.conj() for x,y in zip(prob_list, psi_list))
# mat0 = np.einsum(prob_list, [0], psi_list, [0,1], psi_list.conj(), [0,2], [1,2], optimize=True)

Let's verify that the output matrix `mat0` is a valid density matrix: positive semidefinite and trace 1.

In [ ]:
EVL = np.linalg.eigvalsh(mat0)
print('eigenvalues of mat0:', np.around(EVL, 3))
print('trace of mat0:', np.around(np.trace(mat0), 3))

However, given the density matrix $\rho\in\mathbb{C}^{r\times r}$, finding its pure state decomposition is not that easy. We need the help of Stiefel matrix. Denote the eigenvalue decomposition of $\rho$ as

$$ \rho=\sum_{j}\lambda_{j}|\lambda_{j}\rangle\langle\lambda_{j}| $$

for any Stiefel matrix $X\in\mathrm{St}(r,n)$, we can construct the auxiliary states as (not normalized)

$$ \left|\tilde{\psi}_{i}\right\rangle =\sum_{j=1}^{r}\sqrt{\lambda_{j}}X_{ij}\left|\lambda_{j}\right\rangle. $$

Then, the following pair of quantities

$$ p_{i}=\left\langle \tilde{\psi}_{i}|\tilde{\psi}_{i}\right\rangle ,\left|\psi_{i}\right\rangle =\frac{1}{\sqrt{p_{i}}}\left|\tilde{\psi}_{i}\right\rangle $$

make a valid pure state decomposition of $\rho$. On the other hand, any valid pure state decomposition with $n$ states can be obtained by choosing $X\in\mathrm{St}(r,n)$. See [arxiv-link](https://arxiv.org/abs/2406.19683) for more details. Here, let's verify the statement above numerically.

First, let's generate some random density matrix $\rho$ and calculate its pure state decomposition.

In [ ]:
dim = 3
num_state = 5
rho = numqi.random.rand_density_matrix(dim)
EVL,EVC = np.linalg.eigh(rho)
print('rho=\n', np.around(rho, 3))
print('\neigenvalues=', np.around(EVL, 3))

Then, we randomly generate a Stiefel matrix $X$ and build the auxiliary states as the formula above.

In [ ]:
X = numqi.random.rand_Stiefel_matrix(num_state, dim)
auxiliary_state = X @ (np.sqrt(EVL) * EVC).T

prob_list = np.linalg.norm(auxiliary_state, axis=1)**2
psi_list = auxiliary_state / np.linalg.norm(auxiliary_state, axis=1, keepdims=True)
print('pi:', np.around(prob_list, 3))
print('psi_list.shape:', psi_list.shape)

Let's verify that the decomposition is a valid one $\rho=\sum_i p_i|\psi_i\rangle\langle\psi_i|$.

In [ ]:
mat0 = sum(x*y.reshape(-1,1)*y.conj() for x,y in zip(prob_list, psi_list))
print('matO:\n', np.around(mat0, 3))
print('err(mat0-rho)=', np.abs(mat0-rho).max())
# less than 1e-12, which means the reconstruction is perfect

## Minimization over all pure state decompositions

The power of connecting the pure state decomposition with the Stiefel matrix is that the whole calculation is differentiable, which means we can use the gradient-based optimization method to minimize the convex roof extension. Instead of generating random Stiefel matrix using `numqi.random`, we need to use `numqi.manifold.Stiefel` to support the gradient calculation.

In [ ]:
manifold = numqi.manifold.Stiefel(dim=4, rank=2)
A = manifold()
print('A=', A)
print('\nA^dag A=\n', A.conj().T @ A)


Above, `A` is a Stiefel matirx, also a torch variable with gradient support, which means `A.backward()` is supported. If one have resource function $R$ for any pure state, then the following `DummyModel` can be used to calculate the resource for any density matrix $\rho$.

In [ ]:
def R_for_pure_state(psi):
    # implement the Rourse for a pure state
    raise NotImplementedError


class DummyModel(torch.nn.Module):
    def __init__(self, rho, num_state):
        super().__init__()
        EVL,EVC = np.linalg.eigh(rho)
        self.EVL = torch.tensor(EVL)
        self.EVC = torch.tensor(EVC)
        self.manifold = numqi.manifold.Stiefel(num_state, rho.shape[0])

    def forward(self):
        X = self.manifold()
        auxiliary_state = X @ (torch.sqrt(self.EVL) * self.EVC).T
        prob_list = np.linalg.norm(auxiliary_state, axis=1)**2
        psi_list = auxiliary_state / np.linalg.norm(auxiliary_state, axis=1, keepdims=True)
        loss = sum(x*R_for_pure_state for x,y in zip(prob_list, psi_list))
        return loss

## write your own "R_for_pure_state" function and "rho"
# model = DummyModel(rho, num_state)
# theta_optim = numqi.optimize.minimize(model, theta0='uniform', num_repeat=3, method='L-BFGS-B')


This provide a unified framework for calculating convex roof resource measures. Various resource measures are presented in the following notebooks:

1. [doc-link](https://numqi.github.io/numqi/application/entangle/measure/#entanglement-of-formation) Entanglement of formation
2. [doc-link](https://numqi.github.io/numqi/application/entangle/linear_entropy_entanglement/#linear-entropy-of-entanglement) Lienar entropy of entanglement
3. [doc-link](https://numqi.github.io/numqi/application/magic/stabilizer_purity/#stabilizer-purity) Magic resource: stabilizer purity
4. [doc-link](https://numqi.github.io/numqi/application/coherence/cof/#coherence-of-formation) Cohernce of formation
5. [doc-link](https://numqi.github.io/numqi/application/coherence/gmc/#geometric-measure-of-coherence) Geometric measure of coherence
